In [1]:
library(DESeq2)
library(tidyverse)
library(ggplot2)

Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min



Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomicRanges

Loading required package: GenomeInfoDb

Loading required package: SummarizedExperiment

Loading required package: M

In [2]:
counts = read_csv('../03_build_meta_and_counts/counts.csv') %>% column_to_rownames('gene')
counts %>% head

Rows: 57010 Columns: 186
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr   (1): gene
dbl (185): S0010_01_chow_HY, S0010_02_chow_HY, S0010_03_chow_HY, S0010_04_ch...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


,S0010_01_chow_HY,S0010_02_chow_HY,S0010_03_chow_HY,S0010_04_chow_HY,S0010_05_chow_HY,S0010_06_chow_HY,S0010_01_chow_HPF,S0010_02_chow_HPF,S0010_03_chow_HPF,S0010_04_chow_HPF,⋯,S0038_1424_fast_HPF,S0038_1444_fast_HPF,S0038_1424_fast_SC,S0038_1433_fast_SC,S0038_1434_fast_SC,S0038_1435_fast_SC,S0038_1444_fast_IC,S0038_1453_fast_IC,S0038_1434_fast_ZI,S0038_1454_fast_ZI
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSMUSG00000104478,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENSMUSG00000104385,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENSMUSG00000086053,41,49,37,19,23,22,22,53,34,2,⋯,0,2,0,1,13,4,0,2,0,0
ENSMUSG00000101231,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENSMUSG00000102135,1,5,0,0,0,0,0,0,0,0,⋯,6,3,0,1,2,2,4,11,5,4
ENSMUSG00000103282,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [3]:
meta = read_csv('../04_cleanup/meta_qc.csv') %>% filter(!is.na(name)) %>% column_to_rownames("name") %>% mutate(name = rownames(.))
meta %>% head

Rows: 185 Columns: 26
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr   (9): filename, seq_run, seq_sample, diet, region, hemisphere, notes, n...
dbl  (15): rna_conc_qbit, box_id, animal_id, diet_kcal, experiment_duration,...
dttm  (2): experiment_start, time_of_euthanasia

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


,filename,seq_run,seq_sample,rna_conc_qbit,box_id,animal_id,diet,diet_kcal,region,hemisphere,⋯,bw_change,bw_change_pct,bw_change_pct_ph,fw_i,fw_f,kcal_intake,power_intake,notes,qc,name
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
S0010_01_chow_HY,1_Chow_Hyp_both_S80,0010,S80,73.60,34,1,chow,3227,HY,both,⋯,1.2,0.051282051,0.0032051282,741.7,734.4,11778.55,736.1594,NA,ok,S0010_01_chow_HY
S0010_02_chow_HY,2_Chow_Hyp_both_S44,0010,S44,27.20,34,2,chow,3227,HY,both,⋯,0.1,0.003861004,0.0002388250,741.7,734.4,11778.55,728.5701,NA,ok,S0010_02_chow_HY
S0010_03_chow_HY,3_Chow_Hyp_both_S78,0010,S78,6.36,35,3,chow,3227,HY,both,⋯,0.2,0.007633588,0.0004770992,743.5,735.3,13230.70,826.9188,NA,ok,S0010_03_chow_HY
S0010_04_chow_HY,4_Chow_Hyp_both_S25,0010,S25,20.80,35,4,chow,3227,HY,both,⋯,0.5,0.021276596,0.0013160781,743.5,735.3,13230.70,818.3938,NA,ok,S0010_04_chow_HY
S0010_05_chow_HY,5_Chow_Hyp_both_S84,0010,S84,20.40,36,5,chow,3227,HY,both,⋯,-0.3,-0.012145749,-0.0007591093,713.6,706.4,11617.20,726.0750,NA,ok,S0010_05_chow_HY
S0010_06_chow_HY,6_Chow_Hyp_both_S66,0010,S66,10.60,36,6,chow,3227,HY,both,⋯,1.1,0.042471042,0.0026379529,713.6,706.4,11617.20,721.5652,NA,ok,S0010_06_chow_HY


In [4]:
keep_names = meta %>% filter(qc == 'ok') %>% pull(name)

In [5]:
do_preprocessing = function(counts, meta, keep_names){
    meta_f = meta %>%
        filter(name %in% keep_names)
    counts_f = counts %>%
        select(all_of(keep_names))
    dds <- DESeqDataSetFromMatrix(countData = counts_f,
                          colData = meta_f,
                          design= ~ diet)
    keep <- rowSums(counts(dds)) >= 10
    dds <- dds[keep,]
    dds <- DESeq(dds)
    vsd <- vst(dds, blind=FALSE)
    preprocessed = list(meta_f = meta_f, counts_f = counts_f, dds = dds, vsd=vsd)
    preprocessed
}

In [6]:
preprocessed = do_preprocessing(counts, meta, keep_names)

converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 193 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



In [7]:
vsd_counts = preprocessed %>%
`[[`('vsd') %>%
assay %>%
data.frame %>% 
rownames_to_column('gene')

vsd_counts %>% head
vsd_counts %>% write_csv('vsd_counts.csv')

,gene,S0010_01_chow_HY,S0010_02_chow_HY,S0010_03_chow_HY,S0010_04_chow_HY,S0010_05_chow_HY,S0010_06_chow_HY,S0010_01_chow_HPF,S0010_02_chow_HPF,S0010_03_chow_HPF,⋯,S0037_1454_fast_IC,S0037_1455_fast_IC,S0037_1424_fast_ZI,S0037_1425_fast_ZI,S0037_1433_fast_ZI,S0037_1435_fast_ZI,S0037_1444_fast_ZI,S0037_1445_fast_ZI,S0037_1453_fast_ZI,S0037_1455_fast_ZI
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,ENSMUSG00000086053,5.728104,5.085808,5.126642,5.136153,5.386569,5.522163,5.444345,6.108170,5.816819,⋯,5.684377,5.227758,4.639365,4.563903,4.336926,4.561437,4.808301,6.167270,4.938823,4.306122
2,ENSMUSG00000102135,3.600505,3.805326,3.142339,3.142339,3.142339,3.142339,3.142339,3.142339,3.142339,⋯,3.142339,3.142339,3.803888,3.633226,3.511888,3.873007,3.142339,3.631293,3.758887,3.473212
3,ENSMUSG00000100764,4.150501,4.749909,4.748799,4.567730,4.895106,5.167498,4.847386,5.262716,4.487943,⋯,4.455961,5.303129,4.758015,4.317787,4.228205,4.475958,4.459469,4.695666,4.655855,4.630776
4,ENSMUSG00000100635,4.664551,4.306913,4.649496,4.638712,4.370302,4.372360,4.699287,4.901926,4.290698,⋯,4.086682,4.511476,4.639365,3.633226,4.168661,3.873007,4.525816,4.629267,4.896097,4.347701
5,ENSMUSG00000102213,3.142339,3.440919,3.494014,3.142339,3.142339,3.142339,3.142339,3.142339,3.142339,⋯,3.142339,3.142339,3.142339,3.142339,3.142339,3.142339,3.142339,3.142339,3.142339,3.142339
6,ENSMUSG00000100480,5.137526,5.201422,5.029998,4.939752,5.302195,5.381700,5.354430,5.750047,5.617307,⋯,4.945288,4.970629,5.011021,5.175723,5.474054,4.280285,5.191922,5.512407,5.583775,5.130292


In [8]:
vsd_meta = preprocessed %>%
`[[`('meta_f') %>%
data.frame %>%
tibble %>% 
relocate(name)

vsd_meta %>% head
vsd_meta %>% write_csv('vsd_meta.csv')

name,filename,seq_run,seq_sample,rna_conc_qbit,box_id,animal_id,diet,diet_kcal,region,⋯,bw_f,bw_change,bw_change_pct,bw_change_pct_ph,fw_i,fw_f,kcal_intake,power_intake,notes,qc
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
S0010_01_chow_HY,1_Chow_Hyp_both_S80,0010,S80,73.60,34,1,chow,3227,HY,⋯,24.6,1.2,0.051282051,0.0032051282,741.7,734.4,11778.55,736.1594,NA,ok
S0010_02_chow_HY,2_Chow_Hyp_both_S44,0010,S44,27.20,34,2,chow,3227,HY,⋯,26.0,0.1,0.003861004,0.0002388250,741.7,734.4,11778.55,728.5701,NA,ok
S0010_03_chow_HY,3_Chow_Hyp_both_S78,0010,S78,6.36,35,3,chow,3227,HY,⋯,26.4,0.2,0.007633588,0.0004770992,743.5,735.3,13230.70,826.9188,NA,ok
S0010_04_chow_HY,4_Chow_Hyp_both_S25,0010,S25,20.80,35,4,chow,3227,HY,⋯,24.0,0.5,0.021276596,0.0013160781,743.5,735.3,13230.70,818.3938,NA,ok
S0010_05_chow_HY,5_Chow_Hyp_both_S84,0010,S84,20.40,36,5,chow,3227,HY,⋯,24.4,-0.3,-0.012145749,-0.0007591093,713.6,706.4,11617.20,726.0750,NA,ok
S0010_06_chow_HY,6_Chow_Hyp_both_S66,0010,S66,10.60,36,6,chow,3227,HY,⋯,27.0,1.1,0.042471042,0.0026379529,713.6,706.4,11617.20,721.5652,NA,ok


In [9]:
vsd_meta %>% pull(name)

[1] "S0010_01_chow_HY"    "S0010_02_chow_HY"    "S0010_03_chow_HY"   
  [4] "S0010_04_chow_HY"    "S0010_05_chow_HY"    "S0010_06_chow_HY"   
  [7] "S0010_01_chow_HPF"   "S0010_02_chow_HPF"   "S0010_03_chow_HPF"  
 [10] "S0010_05_chow_HPF"   "S0010_06_chow_HPF"   "S0010_01_chow_SC"   
 [13] "S0010_02_chow_SC"    "S0010_03_chow_SC"    "S0010_04_chow_SC"   
 [16] "S0010_05_chow_SC"    "S0010_02_chow_IC"    "S0010_03_chow_IC"   
 [19] "S0010_04_chow_IC"    "S0010_05_chow_IC"    "S0010_06_chow_IC"   
 [22] "S0010_01_chow_PFC"   "S0010_02_chow_PFC"   "S0010_03_chow_PFC"  
 [25] "S0010_04_chow_PFC"   "S0010_06_chow_PFC"   "S0010_07_fast_HY"   
 [28] "S0010_08_fast_HY"    "S0010_09_fast_HY"    "S0010_10_fast_HY"   
 [31] "S0010_11_fast_HY"    "S0010_07_fast_HPF"   "S0010_08_fast_HPF"  
 [34] "S0010_09_fast_HPF"   "S0010_10_fast_HPF"   "S0010_12_fast_HPF"  
 [37] "S0010_07_fast_SC"    "S0010_08_fast_SC"    "S0010_09_fast_SC"   
 [40] "S0010_11_fast_SC"    "S0010_07_fast_IC"    "S0010_08_fast_IC"   
 [43] "S0010_09_fast_IC"    "S0010_10_fast_IC"    "S0010_11_fast_IC"   
 [46] "S0010_12_fast_IC"    "S0010_07_fast_PFC"   "S0010_08_fast_PFC"  
 [49] "S0010_10_fast_PFC"   "S0010_11_fast_PFC"   "S0010_12_fast_PFC"  
 [52] "S0010_14_HFD_HY"     "S0010_15_HFD_HY"     "S0010_16_HFD_HY"    
 [55] "S0010_17_HFD_HY"     "S0010_18_HFD_HY"     "S0010_13_HFD_HPF"   
 [58] "S0010_14_HFD_HPF"    "S0010_15_HFD_HPF"    "S0010_16_HFD_HPF"   
 [61] "S0010_17_HFD_HPF"    "S0010_18_HFD_HPF"    "S0010_13_HFD_SC"    
 [64] "S0010_14_HFD_SC"     "S0010_15_HFD_SC"     "S0010_16_HFD_SC"    
 [67] "S0010_17_HFD_SC"     "S0010_13_HFD_IC"     "S0010_14_HFD_IC"    
 [70] "S0010_15_HFD_IC"     "S0010_17_HFD_IC"     "S0010_18_HFD_IC"    
 [73] "S0010_13_HFD_PFC"    "S0010_14_HFD_PFC"    "S0010_15_HFD_PFC"   
 [76] "S0010_16_HFD_PFC"    "S0010_17_HFD_PFC"    "S0010_18_HFD_PFC"   
 [79] "S0037_1431_chow_HY"  "S0037_1432_chow_HY"  "S0037_1442_chow_HY" 
 [82] "S0037_1452_chow_HY"  "S0037_1423_chow_HY"  "S0037_1443_chow_HY" 
 [85] "S0037_1422_chow_HPF" "S0037_1432_chow_HPF" "S0037_1442_chow_HPF"
 [88] "S0037_1423_chow_HPF" "S0037_1443_chow_HPF" "S0037_1431_chow_SC" 
 [91] "S0037_1432_chow_SC"  "S0037_1441_chow_SC"  "S0037_1442_chow_SC" 
 [94] "S0037_1451_chow_SC"  "S0037_1452_chow_SC"  "S0037_1423_chow_SC" 
 [97] "S0037_1443_chow_SC"  "S0037_1422_chow_IC"  "S0037_1431_chow_IC" 
[100] "S0037_1432_chow_IC"  "S0037_1441_chow_IC"  "S0037_1451_chow_IC" 
[103] "S0037_1423_chow_IC"  "S0037_1443_chow_IC"  "S0037_1422_chow_ZI" 
[106] "S0037_1431_chow_ZI"  "S0037_1441_chow_ZI"  "S0037_1451_chow_ZI" 
[109] "S0037_1452_chow_ZI"  "S0037_1424_fast_HY"  "S0037_1425_fast_HY" 
[112] "S0037_1433_fast_HY"  "S0037_1434_fast_HY"  "S0037_1444_fast_HY" 
[115] "S0037_1453_fast_HY"  "S0037_1454_fast_HY"  "S0037_1433_fast_HPF"
[118] "S0037_1434_fast_HPF" "S0037_1435_fast_HPF" "S0037_1445_fast_HPF"
[121] "S0037_1453_fast_HPF" "S0037_1454_fast_HPF" "S0037_1455_fast_HPF"
[124] "S0037_1425_fast_SC"  "S0037_1444_fast_SC"  "S0037_1445_fast_SC" 
[127] "S0037_1453_fast_SC"  "S0037_1454_fast_SC"  "S0037_1424_fast_IC" 
[130] "S0037_1425_fast_IC"  "S0037_1433_fast_IC"  "S0037_1434_fast_IC" 
[133] "S0037_1435_fast_IC"  "S0037_1445_fast_IC"  "S0037_1454_fast_IC" 
[136] "S0037_1455_fast_IC"  "S0037_1424_fast_ZI"  "S0037_1425_fast_ZI" 
[139] "S0037_1433_fast_ZI"  "S0037_1435_fast_ZI"  "S0037_1444_fast_ZI" 
[142] "S0037_1445_fast_ZI"  "S0037_1453_fast_ZI"  "S0037_1455_fast_ZI"